## imports

In [1]:
!python -m pip install --upgrade pip
!pip install pyarrow
!pip install pandarallel
!pip install pandas
!pip install mxnet
!pip install autogluon

import numpy as np
import pandas as pd
from pandarallel import pandarallel
import sagemaker
import boto3
import os, time
import autogluon as ag
import mxnet as mx
from mxnet import nd, gluon, init, autograd
from mxnet.gluon import nn
import pickle

os.environ['NUMEXPR_MAX_THREADS'] = str(os.cpu_count())
pandarallel.initialize()
s3_client = boto3.client('s3')

mx.random.seed(127)
contexts = [mx.cpu()]

     |████████████████████████████████| 1.5 MB 10.3 MB/s eta 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 20.0.2
    Uninstalling pip-20.0.2:
      Successfully uninstalled pip-20.0.2
     |████████████████████████████████| 17.2 MB 12.4 MB/s eta 0:00:01
     |████████████████████████████████| 177 kB 8.1 MB/s eta 0:00:01
  Created wheel for pandarallel: filename=pandarallel-1.4.8-py3-none-any.whl size=16111 sha256=c60030d2525f75a475f479c4f4249eea80ba88bba219655d7a9844107ff2d89a
  Stored in directory: /home/ec2-user/.cache/pip/wheels/e9/46/8f/698a358b97ccf5efe84a301297dbf98b19207ff527e0f2512e
  Created wheel for dill: filename=dill-0.3.2-py3-none-any.whl size=78912 sha256=cf8d9528c599346297600ef1c915a12a7663ed4d3e1441fbcdf7a5cf5d50f6fb
  Stored in directory: /home/ec2-user/.cache/pip/wheels/02/49/cf/660924cd9bc5fcddc3a0246fe39800c83028d3ccea244de352
Successfully built pandarallel dill
     |████████████████████████████████| 68.7 MB 97 kB/s s eta 0:00:01     |█

## load data

In [2]:
test_dataset_path = 's3://amlc-data/multimodal-alg-test/test.csv'
model_path = './opt/ml/model-big/'

In [4]:
# text/numeric data
#dataset = pd.read_csv(train_dataset_path)
test_dataset = pd.read_csv(test_dataset_path, header=None)
print('data read')

data read


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (1,9,10,11,12,13,14,15,16,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
model_config = {}

with open(model_path+'model_config.pkl', 'rb') as f:
    model_config = pickle.load(f)
columns = model_config['columns']
columns_text = model_config['columns_text']
columns_num = model_config['columns_num']
columns_cat = model_config['columns_cat']
columns_img = model_config['columns_img']
has_text = model_config['has_text']
has_num = model_config['has_num']
has_cat = model_config['has_cat']
has_img = model_config['has_img']
catmap = model_config['catmap']
invcatmap = model_config['invcatmap']
numcats = model_config['numcats']
cats = model_config['cats']
target_column = 'label'

#columns = columns[1:]
print(columns)
columns.remove('label')
print(columns)
test_dataset.columns = columns

['img_image', 'label', 'marketplace_id_num', 'item_name_text', 'product_description_text', 'bullet_point_text', 'brand_text', 'manufacturer_text', 'generic_keyword_text', 'material_text', 'number_of_items_num', 'case_pack_quantity_num', 'item_package_quantity_num', 'item_dimensions_height_num', 'item_dimensions_width_num', 'item_dimensions_length_num', 'normalized_item_weight_num', 'normalized_item_package_weight_num', 'list_price_currency_cat', 'list_price_value_num', 'list_price_value_with_tax_num', 'ID']
['img_image', 'marketplace_id_num', 'item_name_text', 'product_description_text', 'bullet_point_text', 'brand_text', 'manufacturer_text', 'generic_keyword_text', 'material_text', 'number_of_items_num', 'case_pack_quantity_num', 'item_package_quantity_num', 'item_dimensions_height_num', 'item_dimensions_width_num', 'item_dimensions_length_num', 'normalized_item_weight_num', 'normalized_item_package_weight_num', 'list_price_currency_cat', 'list_price_value_num', 'list_price_value_with

In [11]:
test_dataset = test_dataset.iloc[1:,:]

In [12]:
test_dataset

,img_image,marketplace_id_num,item_name_text,product_description_text,bullet_point_text,brand_text,manufacturer_text,generic_keyword_text,material_text,number_of_items_num,...,item_package_quantity_num,item_dimensions_height_num,item_dimensions_width_num,item_dimensions_length_num,normalized_item_weight_num,normalized_item_package_weight_num,list_price_currency_cat,list_price_value_num,list_price_value_with_tax_num,ID
1,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAsICAgICAsICA...,1.0,Greenworks 40V 300W Cordless Power Inverter IV...,NaN,"Ideal for camping, tailgating, disaster prepar...",Greenworks,GreenWorks,40V Power Inverter,NaN,NaN,...,1.0,0.9,3.82,2.76,1.02,1.30513659104,USD,39.99,NaN,9e4f7cbe4cf54efeb9a7669ec6775cfc
2,NaN,1.0,"Bankers Box SmoothMove Wardrobe Moving Boxes, ...",NaN,Designed specifically to move and store hangin...,Bankers Box,Fellowes,Box Mailers,Cardboard,1.0,...,1.0,34.25,20.38,20.38,NaN,16.6008083286,USD,57.52,NaN,5d76c57ac2b94d4ebe0cbd21b7fc5d68
3,NaN,1.0,High Sierra Powerglide Lightweight Wheeled Lap...,NaN,"LARGE, MULTI-COMPARTMENT DESIGN: this student ...",High Sierra,High Sierra Sport Company,backpack,600D Polyester Waffle Weave,1.0,...,1.0,14.0,9.0,21.0,6.0,6.60064012428,USD,79.99,NaN,94eef78c1be74373a63cc11fa0d34c58
4,NaN,1.0,"Dynamite Screwdriver: #00 Phillips, DYN2826","Dynamite Screwdriver: #00 Phillips, DYN2826","The package height is 0.779""",Dynamite,Dynamite,Other Accessories,NaN,1.0,...,1.0,0.78,8.0,3.6,0.05,0.050706320260000014,USD,11.2,NaN,dadffba3fbf64885ab723e052fce646c
5,NaN,1.0,"Elite Screens Yard Master, 100 inch Outdoor Pr...",NaN,"100-inch Diagonal, 16:9 Aspect Ratio. View Siz...",Elite Screens,Elite Screens Inc.,DaLite Projector Screen Theater White,NaN,1.0,...,1.0,83.9,1.3,99.6,32.2,37.05088775172,USD,263.0,NaN,f3bd48869f484f70a573908b69b10d81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73056,NaN,1,Disney Cars 2 Spies Pull 'n' Race Die Cast Set,A notorious spy ring has been rounded up in th...,"Genuine, Original, Authentic Disney Store",Disney Pixar,Disney,diecast,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.976648,USD,39.95,NaN,68d472e6ad8946c9a9d42be832acb4b4
73057,NaN,1,Delixike Magnifying Glass with Light and Stand...,"<p>Calligraphers,clockmakers, jewelers, and mo...","ULTRA BRIGHT MAGNIFYING LAMP FOR READING, WORK...",Delixike,shen zhen de li xi ke ke ji you xian gong si,desktop handsfree table top standing lighted h...,Metal,NaN,...,1,NaN,NaN,NaN,NaN,1.75267,NaN,NaN,NaN,9bdcd9a3f1284bbaabe2acb15227514c
73058,NaN,1,Women's Footed Tights 3 PK Opaque 120D Solid C...,Women's durable footed tights are the ideal co...,"3 Pairs of High Quality Microfiber, Medium Thi...",FOMANN,NaN,women' tights soft solid pantyhose tights fo...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.3125,NaN,NaN,NaN,b035fbeaa74345ebb6681e08c555559f
73059,NaN,1,Cuckoo Electric Induction Heating Pressure Ric...,NaN,"Essential: For perfectly cooked rice, bread, s...",Cuckoo,Cuckoo,Cuckoo Electronics 10 Cup Stainless IH Pressur...,Stainless Steel,10,...,1,11.4,16.2,11.9,17.4,18.1,USD,519.99,NaN,7dc2e157c48e43d18956f59e7a1f9b5f


## check data

In [11]:
# check for same columns
for i in range(1, len(columns)):
    if not columns[i] in test_dataset.columns and columns[i] != 'label':
        print('big fail!', columns[i])
        exit(0)
print('columns checked')
for i in range(1, len(test_dataset.columns)):
    if not test_dataset.columns[i] in columns and test_dataset.columns[i] != 'label':
        print('big fail!', test_dataset.columns[i])
        exit(0)
print('columns double checked')

columns checked
columns double checked


## data clean

In [13]:
import re, string, base64
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from io import StringIO, BytesIO
from html.parser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        super().__init__()
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.text = StringIO()
    def handle_data(self, d):
        self.text.write(d)
    def get_data(self):
        return self.text.getvalue()
def clean_text(data, labelled=False):
    lemmatizer = WordNetLemmatizer()
    nltk.download('wordnet')
    nltk.download('stopwords')
    stop_words = set(stopwords.words('english'))
    # scrub possible html
    # stackoverflow.com/questions/753052/strip-html-from-strings-in-python
    def strip_tags(html):
        s = MLStripper()
        s.feed(html)
        return s.get_data()
    # lowercases and removes special characters
    def clean_val(val):
        text = val
        text = strip_tags(text)
        if type(val) == str:
            text = text.lower().strip()
            text = re.compile(r'[%s]' % re.escape(string.punctuation)).sub(' ', text)
            text = re.sub(r'\s+', ' ', text)
            words = [w for w in text.split(" ") if not w in stop_words]
            text = " ".join([lemmatizer.lemmatize(w) for w in words])
        return text
    # get text cols
    text_cols = [x for x in data.columns if x.endswith('_text')]
    if len(text_cols) == 0:
        print('no text columns found.')
        return pd.DataFrame(), None
    print('text columns found:', text_cols)
    text_data = data[text_cols].copy()
    # lazy impute text
    text_data = text_data.fillna('')
    # clean text cols
    for col in text_cols:
        print('text cleaning:', col)
        text_data[col] = text_data[col].parallel_apply(clean_val)
    text_label = None
    if labelled:
        print('working on labels...')
        text_label = mappedlabels.copy()
        map2bad = text_data[text_cols[0]] == ''
        for col in text_cols[1:]:
            map2bad = np.logical_and(map2bad, text_data[col] == '')
        text_label[map2bad] = numcats
    print("done transforming text data.")
    return text_data, text_label
def clean_num(data, labelled=False):
    # get num cols
    num_cols = [x for x in data.columns if x.endswith('_num')]
    if len(num_cols) == 0:
        print('no numeric columns found.')
        return pd.DataFrame(), None
    print('numeric columns found:', num_cols)
    num_data = data[num_cols].copy()
    # impute numeric data
    num_data = num_data.fillna(0)
    num_label = None
    if labelled:
        print('working on labels...')
        num_label = mappedlabels.copy()
    print("done transforming numeric data.")
    return num_data, num_label
def clean_cat(data, labelled=False):
    # get cat cols
    cat_cols = [x for x in data.columns if x.endswith('_cat')]
    if len(cat_cols) == 0:
        print('no categorical columns found.')
        return pd.DataFrame(), None
    print('categorical columns found:', cat_cols)
    cat_data = data[cat_cols].copy()
    # impute categorical data
    cat_data = cat_data.fillna('unknown')
    cat_data_out = pd.get_dummies(cat_data)
    cat_label = None
    if labelled:
        print('working on labels...')
        cat_label = mappedlabels.copy()
    print("done transforming categorical data.")
    return cat_data_out, cat_label
def clean_image(data, labelled=False):
    import cv2
    # normalize image
    def normalizeimg(img):
        img = img.transpose((2, 0, 1)).expand_dims(axis=0)
        rgb_mean = nd.array([0.485, 0.456, 0.406]).reshape((1,3,1,1))
        rgb_std = nd.array([0.229, 0.224, 0.225]).reshape((1,3,1,1))
        return (img.astype('float32') / 255 - rgb_mean) / rgb_std
    def cleanimg(img_bytes):
        if img_bytes == '':
            return normalizeimg(mx.nd.array([[[0]*3]*224]*244))
        img = mx.image.imdecode(base64.b64decode(img_bytes))
        img = mx.image.resize_short(img, 256)
        img, _ = mx.image.center_crop(img, (224, 224))
        return normalizeimg(img)
    # get pretrained resnet50
    def getresnet():
        net = gluon.model_zoo.vision.resnet50_v1(pretrained=True, ctx=contexts)
        print('image pre-model created...')
        return net
    # get cat cols
    img_cols = [x for x in data.columns if x.endswith('_image')]
    if len(img_cols) == 0:
        print('no image columns found.')
        return pd.DataFrame(), None
    print('image columns found:', img_cols)
    img_data = data[img_cols].copy()
    # impute categorical data
    img_data = img_data.fillna('')
    # initialize model to transform images to resnet outputs
    img_model = getresnet()
    # create suggested img labels
    images_out = []
    map2bads =  []
    for col in img_cols:
        map2bads.append(img_data[col] == '')
        #temp_col = pd.DataFrame(img_data[col].parallel_apply(cleanimg))
        #print(temp_col.head())
        #print(temp_col.iloc[0, :])
        #print(type(temp_col.iloc[0, :]))
        #print(nd.array(temp_col.iloc[0, :].values))
        #temp_col = temp_col.parallel_apply(nd.array)
        #images_out.append(temp_col.parallel_apply(img_model))
        img_data[col] = img_data[col].parallel_apply(cleanimg).parallel_apply(img_model)
    image_label = []
    if labelled:
        print('working on labels...')
        import functools
        mlabs = mappedlabels.copy()
        for i in range(len(map2bads)):
            image_label.append(mlabs.copy())
            image_label[i][map2bads[i]] = numcats
    print("done transforming image data.")
    #return images_out, image_label
    return img_data, image_label

In [ ]:
text_data, _ = clean_text(test_dataset)
num_data, _ = clean_num(test_dataset)
cat_data, _ = clean_cat(test_dataset)
img_data, _ = clean_image(test_dataset)

[nltk_data] Downloading package wordnet to /home/ec2-user/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


text columns found: ['item_name_text', 'product_description_text', 'bullet_point_text', 'brand_text', 'manufacturer_text', 'generic_keyword_text', 'material_text']
text cleaning: item_name_text


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


text cleaning: product_description_text
text cleaning: bullet_point_text
text cleaning: brand_text
text cleaning: manufacturer_text
text cleaning: generic_keyword_text
text cleaning: material_text
done transforming text data.
numeric columns found: ['marketplace_id_num', 'number_of_items_num', 'case_pack_quantity_num', 'item_package_quantity_num', 'item_dimensions_height_num', 'item_dimensions_width_num', 'item_dimensions_length_num', 'normalized_item_weight_num', 'normalized_item_package_weight_num', 'list_price_value_num', 'list_price_value_with_tax_num']
done transforming numeric data.
categorical columns found: ['list_price_currency_cat']
done transforming categorical data.
image columns found: ['img_image']


Model file not found. Downloading to /home/ec2-user/.mxnet/models/resnet50_v1-0aee57f9.params.


image pre-model created...


## load models

### phase one - per-modality training

In [ ]:
from autogluon import TabularPrediction as task

if has_text:
    print('loading text data')
    agluon_text_train_data = task.Dataset(text_data)
if has_num:
    print('loading num data')
    agluon_num_train_data = task.Dataset(num_data)
if has_cat:
    print('loading cat data')
    agluon_cat_train_data = task.Dataset(cat_data)
if has_img:
    print('loading img data')
    agluon_img_train_data = []
    for i in range(len(img_data.columns)):
        from itertools import zip_longest
        curr_img_feature = pd.DataFrame.from_records(zip_longest(
            *img_data.iloc[:, i].parallel_apply(lambda x: x[0].asnumpy()).values)).transpose()
        agluon_img_train_data.append(task.Dataset(curr_img_feature))
print('done')

loading text data
loading num data
loading cat data
loading img data


In [ ]:
# train on each existing modality
if has_text:
    predictor_text = task.load(model_path+'text/')
    print('done loading text model')
if has_num:
    predictor_num = task.load(model_path+'num/')
    print('done loading num model')
if has_cat:
    predictor_cat = task.load(model_path+'cat/')
    print('done loading cat model')
if has_img:
    predictor_img = []
    for i in range(len(agluon_img_train_data)):
        task.load(model_path+'img/'+img_data.columns[i])
    print('done loading img model')

In [ ]:
if has_text:
    preds_text = predictor_text.predict_proba(agluon_text_train_data)
    print('done generating text unimodal preds')
if has_num:
    preds_num = predictor_num.predict_proba(agluon_num_train_data)
    print('done generating num unimodal preds')
if has_cat:
    preds_cat = predictor_cat.predict_proba(agluon_cat_train_data)
    print('done generating cat unimodal preds')
if has_img:
    preds_img = []
    for i in range(len(predictor_img)):
        preds_img.append(predictor_img[i].predict_proba(agluon_img_train_data[i]))
    print('done generating img unimodal preds')

done generating num unimodal preds


ValueError: Required columns are missing from the provided dataset. Missing columns: ['list_price_currency_cat_GBP']

### phase 2 - wholistic training

In [ ]:
# create data
wholistic_test = pd.DataFrame()
if has_text:
    wholistic_test = pd.concat([wholistic_test, pd.DataFrame(preds_text)], axis=1)
if has_num:
    wholistic_test = pd.concat([wholistic_test, pd.DataFrame(preds_num)], axis=1)
if has_cat:
    wholistic_test = pd.concat([wholistic_test, pd.DataFrame(preds_cat)], axis=1)
if has_img:
    wholistic_test = pd.concat([wholistic_test, pd.DataFrame(preds_img)], axis=1)
wholistic_input_size = wholistic_test.shape[1]

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

In [ ]:
def create_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(
        int(wholistic_input_size*float(2/3))+numcats,
        input_shape=(wholistic_input_size,),
        activation='relu')
    )
    model.add(layers.Dense(int(wholistic_input_size*float(1/3))+numcats, activation='relu'))
    model.add(layers.Dense(numcats, activation='sigmoid'))
    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), 
        optimizer='adam', metrics=['accuracy'])
    return model
model = create_model()

In [ ]:
model.load_weights('./opt/ml/model/wholistic/wholistic_model')
print('model loaded. predicting...')
preds = model.predict(wholistic_test.values)
print('done')

In [ ]:
def getmaplabs(pred):
    return pred.tolist().index(max(pred))

In [ ]:
def unmap(mappedlab):
    return invcatmap[mappedlab]

In [ ]:
predlabs = pd.DataFrame(preds).parallel_apply(getmaplabs, axis=1)
predlabs = predlabs.apply(unmap)

In [ ]:
submission = test_dataset[[id_column]]
submission[label] = predlabs
submission.to_csv('./Submissions/ptc-submission_container-alg.csv', index=False)